# Hidden linear function problem: resource estimation example

In [ ]:
%pip install qbraid-qir \
             cirq-core \
             numpy \
             azure-quantum \
             'azure-quantum[qiskit]' \
             'qiskit<1.0.0'


In [1]:
from typing import List, Tuple, Any

import cirq
import numpy as np
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider
from azure.quantum.qiskit.job import AzureQuantumJob
from qbraid_qir.cirq import cirq_to_qir
from qiskit.tools.monitor import job_monitor

Class for defining the Hidden Linear Function Problem. More information about the problem can be found here: https://quantumai.google/cirq/experiments/hidden_linear_function

In [2]:
class HiddenLinearFunctionProblem:
    """
    Instance of Hidden Linear Function problem.

    The problem is defined by a matrix A and a vector b, which are
    the coefficients of a quadratic form, in which a linear function
    is "hidden". The goal is to find binary vectors `z` that solve the
    problem defined by the quadratic form.

    Attributes:
        A (np.ndarray): Coefficient matrix for the quadratic form.
        b (np.ndarray): Coefficient vector for the quadratic form.
        n (int): Dimensionality of the problem, derived from A's shape.
        L (list of np.ndarray): List of binary vectors in the subspace L_q,
            to which the domain of the quadratic form is restricted.
        all_zs (list of np.ndarray): All binary vectors `z` that are solutions to the problem.

    Raises:
        AssertionError: If A is not an upper triangular matrix, or if b's shape
            does not match the expected (n,).
    """

    def __init__(self, A: np.ndarray, b: np.ndarray):
        """
        Initializes the Hidden Linear Function problem with matrix A and vector b.

        Args:
            A (np.ndarray): Coefficient matrix for the quadratic form, must be upper triangular.
            b (np.ndarray): Coefficient vector for the quadratic form.
        """
        self.n = A.shape[0]
        assert A.shape == (self.n, self.n)
        assert b.shape == (self.n,)
        for i in range(self.n):
            for j in range(i + 1):
                if A[i][j] != 0:  # A[i][j] == 1 iff i < j
                    raise ValueError("A must be an upper triangular matrix")

        self.A = A
        self.b = b
        self.L = None
        self.all_zs = None

    def q(self, x: np.ndarray) -> int:
        """
        Computes the action of the quadratic form on a binary vector (modulo 4).

        Args:
            x (np.ndarray): A binary vector on which the quadratic form acts.

        Returns:
            int: The result of the quadratic form acting on `x`, modulo 4.

        Raises:
            ValueError: If the shape of `x` does not match the expected (n,).
        """
        if x.shape == (self.n,):
            raise ValueError("x must be a binary vector of length n")
        return (2 * (x @ self.A @ x) + (self.b @ x)) % 4

    def bruteforce_solve(self) -> None:
        """
        Finds all binary vectors `z` which are solutions to the problem by brute force.

        This method updates the `L` attribute with vectors in the subspace L_q and
        the `all_zs` attribute with all solution vectors `z`.

        Raises:
            AssertionError: If called before `q` has been properly defined.
        """
        # All binary vectors of length `n`.
        all_vectors = [np.array([(m >> i) % 2 for i in range(self.n)]) for m in range(2**self.n)]

        def vector_in_L(x):
            for y in all_vectors:
                if self.q((x + y) % 2) != (self.q(x) + self.q(y)) % 4:
                    return False
            return True

        self.L = [x for x in all_vectors if vector_in_L(x)]
        self.all_zs = [z for z in self.L if self.is_z(z)]

    def is_z(self, z: np.ndarray) -> bool:
        """
        Checks whether a given vector `z` is a solution to the problem.

        Args:
            z (np.ndarray): The binary vector to check.

        Returns:
            bool: True if `z` is a solution, False otherwise.

        Raises:
            ValueError: If `z`'s shape does not match the expected (n,)
            RuntimeError: If `L` has not been computed yet.
        """
        if z.shape == (self.n,):
            raise ValueError("z must be a binary vector of length n")

        if self.L is None:
            raise RuntimeError("L has not been computed yet, it must be")

        for x in self.L:
            if self.q(x) != 2 * ((z @ x) % 2):
                return False
        return True

Example problem which is hard classically

In [3]:
A = np.array(
    [
        [0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
)
b = np.array([0, 0, 0, 0, 1, 1, 1, 0, 0, 1])
problem_10_64 = HiddenLinearFunctionProblem(A, b)

In [4]:
def edge_coloring(A: np.ndarray) -> List[List[Tuple[int, int]]]:
    """Solves the edge coloring problem for a given graph represented by its adjacency matrix.

    This function attempts to find an edge coloring of the graph such that no two edges sharing
    a common vertex have the same color. The goal is to minimize the number of colors used.

    Args:
        A (np.ndarray): The adjacency matrix of the graph, where `A[i, j] == 1` indicates an edge
                        between vertices `i` and `j`, and `A[i, j] == 0` indicates no edge.

    Returns:
        List[List[Tuple[int, int]]]: A list of edge groups, where each group contains edges (as tuples
                                     of vertices `(i, j)`) that can be assigned the same color without
                                     violating the coloring constraints. Edges in the same group do not
                                     share a common vertex.
    """
    A = np.copy(A)
    n = A.shape[0]
    ans: List[List[Tuple[int, int]]] = []
    while np.max(A) != 0:
        edges_group: List[Tuple[int, int]] = []
        used = np.zeros(n, dtype=bool)
        for i in range(n):
            for j in range(i + 1, n):  # Adjusted to avoid duplication and ensure i < j
                if A[i][j] == 1 and not used[i] and not used[j]:
                    edges_group.append((i, j))
                    A[i][j] = A[j][i] = 0  # Ensure the edge is removed from both [i, j] and [j, i]
                    used[i] = used[j] = True
        ans.append(edges_group)
    return ans

In [5]:
def generate_circuit_for_problem(problem: HiddenLinearFunctionProblem) -> cirq.Circuit:
    """
    Generates a quantum circuit using Cirq that solves an instance of the Hidden Linear Function problem.

    The generated circuit includes Hadamard gates to create an equal superposition of all states, controlled-Z
    gates to encode the matrix A of the problem, S gates to encode the vector b, another set of Hadamard gates,
    and finally measurement gates.

    Args:
        problem (HiddenLinearFunctionProblem): An instance of the Hidden Linear Function problem, defined by
                                               a symmetric binary matrix A and a binary vector b.

    Returns:
        cirq.Circuit: The quantum circuit designed to solve the given instance of the Hidden Linear Function
                      problem. The circuit applies a series of quantum gates to qubits arranged in a line
                      and measures each qubit at the end.
    """

    qubits = cirq.LineQubit.range(problem.n)
    circuit = cirq.Circuit()

    # Hadamard gates at the beginning (creating equal superposition of all states).
    circuit += cirq.Moment([cirq.H(q) for q in qubits])

    # Controlled-Z gates encoding the matrix A.
    for layer in edge_coloring(problem.A):
        for i, j in layer:
            circuit += cirq.CZ(qubits[i], qubits[j])

    # S gates encoding the vector b.
    circuit += cirq.Moment([cirq.S.on(qubits[i]) for i in range(problem.n) if problem.b[i] == 1])

    # Hadamard gates at the end.
    circuit += cirq.Moment([cirq.H(q) for q in qubits])

    # Measurements.
    circuit += cirq.Moment([cirq.measure(qubits[i], key=str(i)) for i in range(problem.n)])

    return circuit

Instatiating the AzureQuantumProvider with resource information

In [6]:
workspace = Workspace(
    resource_id="",  # Your resource_id
    location="westus",  # Your workspace location (for example, "westus")
)
provider = AzureQuantumProvider(workspace)

In [7]:
def resource_estimation_job_from_qir(
    provider: AzureQuantumProvider, bitcode: bytes, **kwargs: Any
) -> AzureQuantumJob:
    """
    Creates a resource estimation job from QIR bitcode using an Azure Quantum Provider.

    Args:
        provider (AzureQuantumProvider): The Azure Quantum Provider through which the job
            will be submitted.
        bitcode (bytes): The QIR bitcode that defines the quantum operation for which
            resources are to be estimated.
        **kwargs (Any): Additional keyword arguments for job configuration. This may include
            'name' for the job name and other backend-specific settings.

    Returns:
        AzureQuantumJob: The initialized Azure Quantum job object ready for submission.
    """

    # Find the Resource Estimator target from the provider
    backend = provider.get_backend("microsoft.estimator")

    # Job name handling via keyword arguments
    name = kwargs.pop("name", "QIR job")

    # Extract job-specific arguments from the backend's configuration
    config = backend.configuration()
    blob_name = config.azure["blob_name"]
    content_type = config.azure["content_type"]
    provider_id = config.azure["provider_id"]
    output_data_format = config.azure["output_data_format"]

    # Create and return the Azure Quantum job object
    return AzureQuantumJob(
        backend=backend,
        target=backend.name(),
        name=name,
        input_data=bitcode,
        blob_name=blob_name,
        content_type=content_type,
        provider_id=provider_id,
        input_data_format="qir.v1",
        output_data_format=output_data_format,
        input_params=kwargs,
        metadata={},
    )

Converting the Cirq circuit to QIR and running resource estimation

In [8]:
problem_circuit = generate_circuit_for_problem(problem_10_64)

print(problem_circuit)

          ┌───┐   ┌──┐   ┌───┐   ┌───┐   ┌───┐
0: ───H────@───────@──────@───────@───────@──────────────────────H───M('0')───
           │       │      │       │       │
1: ───H────@───────┼@─────┼@──────┼@──────┼@─────@───@───@───────H───M('1')───
                   ││     ││      ││      ││     │   │   │
2: ───H────@───────@┼─────┼┼@─────┼┼@─────┼┼─────┼───┼───┼───────H───M('2')───
           │        │     │││     │││     ││     │   │   │
3: ───H────┼@───────@─────┼┼┼─────┼┼┼─────┼┼─────┼───┼───┼───────H───M('3')───
           ││             │││     │││     ││     │   │   │
4: ───H────┼┼@─────@──────┼@┼─────┼┼┼─────┼┼─────┼───┼───┼───S───H───M('4')───
           │││     │      │ │     │││     ││     │   │   │
5: ───H────┼┼@─────┼@─────@─┼─────┼@┼─────┼┼@────┼───┼───┼───S───H───M('5')───
           ││      ││       │     │ │     │││    │   │   │
6: ───H────@┼──────┼@─────@─┼─────┼─┼─────┼@┼────┼───┼───┼───S───H───M('6')───
            │      │      │ │     │ │     │ │    │   │   │
7:

In [9]:
module = cirq_to_qir(problem_circuit, record_output=False, name="problem_module")

job = resource_estimation_job_from_qir(provider, module.bitcode, errorBudget=0.05)

job_monitor(job)

Job Status: job has successfully run


In [10]:
job.result()

Runtime,20 microsecs,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (2,000 nanosecs) multiplied by the 10 logical cycles to run the algorithm. If however the duration of a single T factory (here: 0 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,15.00M,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (30) (with a logical error rate of 1.67e-4) multiplied by the clock frequency (500,000.00), which is the number of logical cycles per second."
Physical qubits,1.50k,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 1,500 physical qubits to implement the algorithm logic, and 0 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,30,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 10$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 30$ logical qubits."
Algorithmic depth,10,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 10 single-qubit measurements, the 0 arbitrary single-qubit rotations, and the 0 T gates, three multi-qubit measurements for each of the 0 CCZ and 0 CCiX gates in the input program, as well as No rotations in algorithm multi-qubit measurements for each of the 0 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,10,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 10. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,500.00k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 2 microsecs.
Number of T states,0,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 0 T gates, four T states for each of the 0 CCZ and 0 CCiX gates, as well as No rotations in algorithm for each of the 0 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,0,"Number of T factories capable of producing the demanded 0 T states during the algorithm's runtime The total number of T factories 0 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{0 \cdot 0\;\text{ns}}{0 \cdot 20,000\;\text{ns}}\right\rceil$"
Number of T factory invocations,0,"Number of times all T factories are invoked In order to prepare the 0 T states, the 0 copies of the T factory are repeatedly invoked 0 times."
Physical algorithmic qubits,1.50k,"Number of physical qubits for the algorithm after layout The 1,500 are the product of the 30 logical qubits after layout and the 50 physical qubits that encode a single logical qubit."
